In [1]:
from mne.io import concatenate_raws, read_raw_edf
import mne
import glob

In [2]:
rest_path = r"../patient_data/静息脑电（第一批次）/"
therapy_path=r"../patient_data/治疗脑电（第一批次）/"

# rest_subdirectory = sorted(glob.glob(rest_path+"*"))
# therapy_subdirectory = sorted(glob.glob(therapy_path+"*"))
therapy_file_path = sorted(glob.glob(therapy_path+"*"))

In [3]:
# therapy_file_path

In [4]:
# print(rest_subdirectory)
# # 一级子目录
# print(therapy_subdirectory)

In [5]:
# therapy_file_path = []
# for directory in therapy_subdirectory:
#     therapy_file_path.append(glob.glob(directory+"/*"))
# # file_path

In [6]:
#检测EEG文件是否有flag标记,返回true/false
def judgeflag(file_path,flag):
    raw = read_raw_edf(file_path,preload=False)
    events_from_annot, event_dict = mne.events_from_annotations(raw)
    if flag in event_dict:
        return True
    else:
        return False

In [7]:
# 数据采样率1024Hz,文件路径；采集时间；麻醉起始点的偏移(s)
def get_se_epochs(path,select_time,bias):
    #选择15个通道
    chs1 = ['EEG Fp1-AV', 'EEG Fp2-AV', 'EEG F3-AV', 'EEG F4-AV', 'EEG C3-AV', 'EEG C4-AV', 'EEG P3-AV', 'EEG P4-AV',
       'EEG O1-AV', 'EEG O2-AV', 'EEG T3-AV', 'EEG T4-AV', 'EEG T5-AV', 'EEG T6-AV','EEG Cz-AV']
    chs2 =['EEG Fp1-A1', 'EEG Fp2-A2', 'EEG F3-A1','EEG F4-A2','EEG C3-A1','EEG C4-A2','EEG P3-A1','EEG P4-A2','EEG O1-A1',
           'EEG O2-A2','EEG T3-A1','EEG T4-A2','EEG T5-A1','EEG T6-A2','EEG Cz-AAV']
    # select_time 采集的时间
    raw=read_raw_edf(path,preload=False)
    if (chs1[0] in raw.info.ch_names):
        chs=chs1
    else:
        chs=chs2
    raw=raw.pick_channels(chs)
    events_from_annot, event_dict = mne.events_from_annotations(raw)
    # 第二个标记是SE，麻醉标记
    start_time=float(events_from_annot[2][0])/1024 + bias
    # SE标记处,save函数的参数单位是s，所以要除以1024
    end_time=start_time+select_time
    print(start_time,"+",end_time)
    #选择所有的15通道
    data,time = raw[:,:] 
    picks=None
    path_name=path.split('\\')[1:]
    path_name="".join(path_name)
    path_name=path_name.split('.')[0]
    path_name=r"../SE_train/" + path_name + "_bias:" + str(bias) +".fif"
    path_name=path_name.replace('/','\\')
    print(path_name)
    # 文件保存至当前目录下的SE文件夹中
    raw.save(path_name,picks,start_time,end_time,overwrite=True)

In [9]:
# 根据是否有SE标签进行切分
# file_flag_list = []
# for floder in therapy_file_path:
#         for file_path in floder:
#             if judgeflag(file_path,"SE"):
#                 file_flag_list.append(1)
#             else:
#                 file_flag_list.append(0)

file_without_se = []
for file_path in therapy_file_path:
    if judgeflag(file_path,"SE"):
        get_se_epochs(file_path,8,0)
        # 如果是未发作的样本，采集四倍
        if file_path.find('未发作') != -1:
            get_se_epochs(file_path,8,8)
            get_se_epochs(file_path,8,16)
            get_se_epochs(file_path,8,24)
    else:
        file_without_se.append(file_path)

Extracting EDF parameters from H:\Anaconda-jupyter\ECT课题\第四批病人数据\治疗脑电（第一批次）\Z何富桂20220221.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Used Annotations descriptions: ['File Create', 'Recording ends', 'Recording starts', 'SE', 'end', 'start']
Extracting EDF parameters from H:\Anaconda-jupyter\ECT课题\第四批病人数据\治疗脑电（第一批次）\Z何富桂20220221.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Used Annotations descriptions: ['File Create', 'Recording ends', 'Recording starts', 'SE', 'end', 'start']
8.7099609375 + 16.7099609375
.\SE\Z何富桂202202210.fif
Writing H:\Anaconda-jupyter\ECT课题\病人数据处理\SE\Z何富桂202202210.fif
Closing H:\Anaconda-jupyter\ECT课题\病人数据处理\SE\Z何富桂202202210.fif
[done]
Extracting EDF parameters from H:\Anaconda-jupyter\ECT课题\第四批病人数据\治疗脑电（第一批次）\Z何富桂20220225.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


<ipython-input-7-ed7de2df3edd>:21: RuntimeWarning: This filename (H:\Anaconda-jupyter\ECT课题\病人数据处理\SE\Z何富桂202202210.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw.save(path_name,picks,start_time,end_time,overwrite=True)


Used Annotations descriptions: ['File Create', 'Recording ends', 'Recording starts', 'SE', 'end', 'start']
Extracting EDF parameters from H:\Anaconda-jupyter\ECT课题\第四批病人数据\治疗脑电（第一批次）\Z何富桂20220225.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Used Annotations descriptions: ['File Create', 'Recording ends', 'Recording starts', 'SE', 'end', 'start']
227.634765625 + 235.634765625
.\SE\Z何富桂202202250.fif
Writing H:\Anaconda-jupyter\ECT课题\病人数据处理\SE\Z何富桂202202250.fif
Closing H:\Anaconda-jupyter\ECT课题\病人数据处理\SE\Z何富桂202202250.fif
[done]
Extracting EDF parameters from H:\Anaconda-jupyter\ECT课题\第四批病人数据\治疗脑电（第一批次）\Z何富桂20220228.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Used Annotations descriptions: ['File Create', 'Recording ends', 'Recording starts', 'SE', 'end', 'start']
Extracting EDF parameters from H:\Anaconda-jupyter\ECT课题\第四批病人数据\治疗脑电（第一批次）\Z何富桂20220228.edf...
EDF file detected
Setting channel info st

<ipython-input-7-ed7de2df3edd>:21: RuntimeWarning: This filename (H:\Anaconda-jupyter\ECT课题\病人数据处理\SE\Z何富桂202202250.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw.save(path_name,picks,start_time,end_time,overwrite=True)
<ipython-input-7-ed7de2df3edd>:21: RuntimeWarning: This filename (H:\Anaconda-jupyter\ECT课题\病人数据处理\SE\Z何富桂202202280.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw.save(path_name,picks,start_time,end_time,overwrite=True)


EDF file detected
Setting channel info structure...
Creating raw.info structure...
Used Annotations descriptions: ['File Create', 'Recording ends', 'Recording starts', 'SE', 'end', 'start']
Extracting EDF parameters from H:\Anaconda-jupyter\ECT课题\第四批病人数据\治疗脑电（第一批次）\Z何富桂20220302.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Used Annotations descriptions: ['File Create', 'Recording ends', 'Recording starts', 'SE', 'end', 'start']
130.6416015625 + 138.6416015625
.\SE\Z何富桂202203020.fif
Writing H:\Anaconda-jupyter\ECT课题\病人数据处理\SE\Z何富桂202203020.fif
Closing H:\Anaconda-jupyter\ECT课题\病人数据处理\SE\Z何富桂202203020.fif
[done]
Extracting EDF parameters from H:\Anaconda-jupyter\ECT课题\第四批病人数据\治疗脑电（第一批次）\Z倪诗瑄202203011.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Used Annotations descriptions: ['File Create', 'Recording ends', 'Recording starts', 'SE', 'end', 'start']
Extracting EDF parameters from H:\Anaconda-jupyter\

...

<ipython-input-7-ed7de2df3edd>:21: RuntimeWarning: This filename (H:\Anaconda-jupyter\ECT课题\病人数据处理\SE\Z齐金钢202202250.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw.save(path_name,picks,start_time,end_time,overwrite=True)


20.4990234375 + 28.4990234375
.\SE\Z齐金钢202202280.fif
Writing H:\Anaconda-jupyter\ECT课题\病人数据处理\SE\Z齐金钢202202280.fif
Closing H:\Anaconda-jupyter\ECT课题\病人数据处理\SE\Z齐金钢202202280.fif
[done]
Extracting EDF parameters from H:\Anaconda-jupyter\ECT课题\第四批病人数据\治疗脑电（第一批次）\Z齐金钢20220302.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Used Annotations descriptions: ['File Create', 'Recording ends', 'Recording starts', 'SE', 'end', 'start']
Extracting EDF parameters from H:\Anaconda-jupyter\ECT课题\第四批病人数据\治疗脑电（第一批次）\Z齐金钢20220302.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Used Annotations descriptions: ['File Create', 'Recording ends', 'Recording starts', 'SE', 'end', 'start']


<ipython-input-7-ed7de2df3edd>:21: RuntimeWarning: This filename (H:\Anaconda-jupyter\ECT课题\病人数据处理\SE\Z齐金钢202202280.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw.save(path_name,picks,start_time,end_time,overwrite=True)


146.841796875 + 154.841796875
.\SE\Z齐金钢202203020.fif
Writing H:\Anaconda-jupyter\ECT课题\病人数据处理\SE\Z齐金钢202203020.fif
Closing H:\Anaconda-jupyter\ECT课题\病人数据处理\SE\Z齐金钢202203020.fif
[done]


<ipython-input-7-ed7de2df3edd>:21: RuntimeWarning: This filename (H:\Anaconda-jupyter\ECT课题\病人数据处理\SE\Z齐金钢202203020.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw.save(path_name,picks,start_time,end_time,overwrite=True)


In [10]:
file_without_se

['../第四批病人数据/治疗脑电（第一批次）\\Z刘家瑞20220304.edf',
 '../第四批病人数据/治疗脑电（第一批次）\\Z张璐20220304.edf',
 '../第四批病人数据/治疗脑电（第一批次）\\Z林攀攀20220221（未发作）.edf',
 '../第四批病人数据/治疗脑电（第一批次）\\Z肖智华20220307（未发作）.edf']

In [12]:
# file_name = [j for i in therapy_file_path for j in i]

In [14]:
# for i in range(len(file_name)):
#     print(file_name[i],":",file_flag_list[i])